<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> COP-4</h1>
</center>

## <span style="color:blue">Nombre</span>: José Miguel Quezada Silva

## <span style="color:blue">Rol</span>: 201773528-7

# Librerías

In [11]:
import numpy as np
#import scipy as sp
import sympy as sym
#from matplotlib import pyplot as plt

# Desarrollo

## 1

In [6]:
data = np.load("1.npy")
x = data[:,0]
y = data[:,1]

In [116]:
eps = 1e-10
lamb = 1

In [49]:
f2 = lambda x, beta: beta[0] + np.sin(beta[1]*x) + np.cos(beta[2]*x)

In [60]:
def buildF(beta, x_i, y_i):
    #beta: arreglo de betas
    #x_i: arreglo de x
    #y_i: arreglo de y
    
    n = x_i.shape[0]
    
    F = np.zeros([3,1])
    
    for i in range(n):
        
        func = (y_i[i] - f2(x_i[i], beta))**3
        
        F[0] += func
        F[1] += x_i[i] * np.cos(beta[1]*x_i[i]) * func
        F[2] += x_i[i] * np.sin(beta[2]*x_i[i]) * func
        
    
    return F

In [246]:
def app_J(beta, x, y):
    #beta: arreglo de betas
    
    J =  np.zeros([3,3])
    
    Fb = buildF(beta, x, y)
    
    J[0] = ((buildF([beta[0] + eps, beta[1], beta[2]], x, y) - Fb) / eps).T
    J[1] = ((buildF([beta[0], beta[1] + eps, beta[2]], x, y) - Fb) / eps).T
    J[2] = ((buildF([beta[0], beta[1], beta[2] + eps], x, y) - Fb) / eps).T
    
    #se transpone el resultado para ingresarlo correctamente a cada fila, para 
    #finalmente retornar la matriz con esos resultados como columnas
    
    return J.T

In [217]:
#initial guess
bi = np.ones(3).T

#### 1.

In [218]:
buildF(bi,x,y)[1][0]

-1332.002885073915

#### 2.

In [219]:
np.linalg.norm(app_J(bi,x,y), ord = 'fro')

16783.575729300683

#### 3

In [254]:
#Levenberg-Marquardt

#initial guess
bi = np.ones(3).T

iteraciones = 4

for it in range(iteraciones):
    
    Fi = buildF(bi, x, y)
    Ji = app_J(bi, x, y)
    
    bi += (np.linalg.solve((Ji.T@Ji + lamb*np.eye(3)), (-Ji.T@Fi))).T[0]


In [256]:
bi[0]

1.029372373117519

In [268]:
#Jacobi

#initial guess
bi = np.ones(3).T

iteraciones = 10

tol = 1e-10

A = Ji.T@Ji + lamb*np.eye(3)
b = -Ji.T@Fi

L = np.tril(A, -1)
U = np.triu(A, 1)
D = np.diag(np.diag(A))

#para obtener error
b_prev = bi

for it in range(iteraciones):
    
    bi = np.linalg.inv(D) @ (b[:,0] - (L + U)@b_prev)
    
    if np.linalg.norm(bi - b_prev) < tol:
        break

    b_prev = bi

bi

array([-0.02678171,  0.00060325,  0.05000367])